# Vertex Pipelines: Qwik Start

Pipelines help you automate and reproduce your ML workflow. Vertex AI integrates the ML offerings across Google Cloud into a seamless development experience. 

Previously, models trained with AutoML and custom models were accessible via separate services. Vertex AI combines both into a single API, along with other new products. Vertex AI also includes a variety of MLOps products, like Vertex Pipelines. 

In this lab, you will learn how to create and run ML pipelines with Vertex Pipelines.

## Why Are ML Pipelines Useful?

Before diving in, first understand why you would want to use a pipeline. 

Imagine you're building out a ML workflow that includes processing data, training a model, hyperparameter tuning, evaluation, and model deployment. Each of these steps may have different dependencies, which may become unwieldy if you treat the entire workflow as a monolith. 

As you begin to scale your ML process, you might want to share your ML workflow with others on your team so they can run it and contribute code. Without a reliable, reproducible process, this can become difficult. 

With pipelines, each step in your ML process is its own container. This lets you develop steps independently and track the input and output from each step in a reproducible way. You can also schedule or trigger runs of your pipeline based on other events in your Cloud environment, like when new training data is available.

## Objectives

* Use the Kubeflow Pipelines SDK to build scalable ML pipelines.
* Create and run a 3-step intro pipeline that takes text input.
* Create and run a pipeline that trains, evaluates, and deploys an AutoML classification model.
* Use pre-built components for interacting with Vertex AI services, provided through the google_cloud_pipeline_components library.
* Schedule a pipeline job with Cloud Scheduler.

# Vertex Pipelines Setup

There are a few additional libraries you'll need to install in order to use Vertex Pipelines:

* **Kubeflow Pipelines**: This is the SDK used to build the pipeline. Vertex Pipelines supports running pipelines built with both Kubeflow Pipelines or TFX.
* **Google Cloud Pipeline Components**: This library provides pre-built components that make it easier to interact with Vertex AI services from your pipeline steps.

## Install Libraries

To install both services needed for this lab, first set the user flag in a notebook cell:

In [ ]:
USER_FLAG = "--user"

In [2]:
!pip3 install {USER_FLAG} google-cloud-aiplatform==1.0.0 --upgrade
!pip3 install {USER_FLAG} kfp google-cloud-pipeline-components==0.1.1 --upgrade

     |████████████████████████████████| 1.8 MB 5.2 MB/s            
     |████████████████████████████████| 93 kB 2.3 MB/s             
     |████████████████████████████████| 152 kB 55.3 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
explainable-ai-sdk 1.3.2 requires xai-image-widget, which is not installed.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.31.0 which is incompatible.
     |████████████████████████████████| 301 kB 4.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 636 kB 76.3 MB/s            
     |████████████████████████████████| 1.6 MB 61.8 MB/s            
     |████████████████████████████████| 62 kB 929

After installing these packages you'll need to restart the kernel before continuing.

In [1]:
import os

# Automatically restart kernel after installs
if not os.getenv("IS_TESTING"):
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

Finally, check that you have correctly installed the packages. The KFP SDK version should be >=1.6:

In [1]:
!python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
!python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 1.8.12
google_cloud_pipeline_components version: 0.1.1


## Step 2: Set Project ID and Bucket

Now, we'll create variables for each Project ID and Bucket.

In [2]:
import os
PROJECT_ID = "qwiklabs-gcp-01-33949882a483"
BUCKET_NAME="gs://" + PROJECT_ID + "-bucket"

## Step 3: Import libraries

Import the libraries will be used throughout this lab:

In [3]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, ClassificationMetrics, Metrics, component)
from kfp.v2.google.client import AIPlatformClient

from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip

## Step 4: Define Constants

The last thing we need to do before building the pipeline is define some constant variables. `PIPELINE_ROOT` is the Cloud Storage path where the artifacts created by our pipeline will be written. We're using `us-central1` as the region here.

In [4]:
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline_root/"
PIPELINE_ROOT

env: PATH=/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


'gs://qwiklabs-gcp-01-33949882a483-bucket/pipeline_root/'

After running the code above, you should see the root directory for your pipeline printed. This is the Cloud Storage location where the artifacts from your pipeline will be written. It will be in the format of `gs://<bucket_name>/pipeline_root/`

# Creating Our First Pipeline

Create a short pipeline using the KFP SDK. This pipeline doesn't do anything ML related, this exercise is to teach us:

* How to create custom components in the KFP SDK.
* How to run and monitor a pipeline in Vertex Pipelines.

We'll create a pipeline that prints out a sentence using two outputs: a product name and an emoji description. This pipeline will consist of three components:

1. `product_name`: This component will take a product name as input, and return that string as output.
2. `emoji`: This component will take the text description of an emoji and convert it to an emoji. 
    * For example, the text code for ✨ is "sparkles". This component uses an emoji library to show you how to manage external dependencies in your pipeline.
3. `build_sentence`: This final component will consume the output of the previous two to build a sentence that uses the emoji. 
    * For example, the resulting output might be "Vertex Pipelines is ✨".

## Step 1: Create A `Python` Function-Based Component

Using the KFP SDK, we can create components based on `Python` functions. First build the `product_name` component, which simply takes a string as input and returns that string.

In [5]:
@component(base_image="python:3.9", output_component_file="first-component.yaml")
def product_name(text: str) -> str:
    return text

Take a closer look at the syntax here:

* The `@component` decorator compiles this function to a component when the pipeline is run. We'll use this anytime we write a custom component.
* The `base_image` parameter specifies the container image this component will use.
* The `output_component_file` parameter is optional, and specifies the `yaml` file to write the compiled component to. 

After running the cell above, we should see that file written to our notebook instance. If we wanted to share this component with someone, we could send them the generated `yaml` file and have them load it with the following:

In [6]:
product_name_component = kfp.components.load_component_from_file('./first-component.yaml')

The `-> str` after the function definition specifies the output type for this component.

## Step 2: Create 2 Additional Components

To complete the pipeline, create two more components. The first one takes a string as input, and converts this string to its corresponding emoji if there is one. It returns a tuple with the input text passed, and the resulting emoji:

In [7]:
@component(packages_to_install=["emoji"])
def emoji(text: str) -> NamedTuple("Outputs", [("emoji_text", str), ("emoji", str)]):
    
    import emoji
    
    emoji_text = text
    emoji_str = emoji.emojize(':' + emoji_text + ':', use_aliases=True)
    
    print("output one: {}; output_two: {}".format(emoji_text, emoji_str))
    
    return (emoji_text, emoji_str)

This component is a bit more complex than the previous one. Here's what's new:

* The `packages_to_install` parameter tells the component any external library dependencies for this container. In this case, we're using a library called emoji.
* This component returns a `NamedTuple` called `Outputs`. Notice that each of the strings in this tuple have keys: `emoji_text` and `emoji`. We'll use these in our next component to access the output.

The final component in this pipeline will consume the output of the first two and combine them to return a string:

In [8]:
@component
def build_sentence(product: str, emoji: str, emojitext: str) -> str:
    
    print("We completed the pipeline, hooray!")
    
    end_str = product + " is "
    end_str += emoji if (len(emoji) > 0) else emojitext
    
    return(end_str)

## Step 3: Putting The Components Together Into A Pipeline

The component definitions defined above created factory functions that can be used in a pipeline definition to create steps. To set up a pipeline, use the `@dsl.pipeline` decorator, give the pipeline a name and description, and provide the root path where our pipeline's artifacts should be written. By artifacts, it means any output files generated by our pipeline. This intro pipeline doesn't generate any, but our next pipeline will.

In the next block of code we will define an `intro_pipeline` function. This is where we specify the inputs to your initial pipeline steps, and how steps connect to each other:

* `product_task` takes a product name as input. Here we're passing "Vertex Pipelines" but we can change this to whatever we'd like.
* `emoji_task` takes the text code for an emoji as input. We can also change this to whatever we'd like. 
    * For example, "party_face" refers to the 🥳 emoji. 
    * Note that since both this and the `product_task` component don't have any steps that feed input into them, we manually specify the input for these when we define our pipeline.
* The last step in the pipeline - `consumer_task` has three input parameters:
    1. The output of `product_task`. Since this step only produces one output, we can reference it via `product_task.output`.
    2. The emoji output of the `emoji_task` step. See the `emoji` component defined above where we named the output parameters.
    3. Similarly, the `emoji_text` named `Output` from the `emoji` component. In case our pipeline passed text that doesn't correspond with an emoji, it'll use this text to construct a sentence.

In [9]:
@dsl.pipeline(name="hello-world",
    description="An intro pipeline",
    pipeline_root=PIPELINE_ROOT
)

# You can change the `text` and `emoji_str` parameters here to update the pipeline output
def intro_pipeline(text: str = "Vertex Pipelines", emoji_str: str = "sparkles"):
    product_task = product_name(text)
    emoji_task = emoji(emoji_str)
    consumer_task = build_sentence(
        product_task.output,
        emoji_task.outputs["emoji"],
        emoji_task.outputs["emoji_text"]
    )

## Step 4: Compile & Run The Pipeline

With our pipeline defined, we're ready to compile it. The following will generate a `JSON` file that we'll use to run the pipeline:

In [10]:
compiler.Compiler().compile(
    pipeline_func=intro_pipeline, package_path="intro_pipeline_job.json"
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1281: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


Next, instantiate an API client:

In [11]:
api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


Finally, run the pipeline:

In [12]:
response = api_client.create_run_from_job_spec(
    job_spec_path="intro_pipeline_job.json",
    # pipeline_root=PIPELINE_ROOT             # this argument is necessary if you did not specify PIPELINE_ROOT as part of the pipeline definition.
)

Running the pipeline should generate a link to view the pipeline run in your console. It should look like this when complete:

![](https://cdn.qwiklabs.com/k5Jzue01zuwdjgGZFEe3OCIcoFjihCpuheBlT2hE2RI%3D)

This pipeline will take 5-6 minutes to run.

# Creating An End-To-End ML Pipeline

It's time to build our first ML pipeline. In this pipeline, we'll use the UCI Machine Learning Dry Beans dataset, from: KOKLU, M. and OZKAN, I.A., (2020), "Multiclass Classification of Dry Beans Using Computer Vision and Machine Learning Techniques." In Computers and Electronics in Agriculture, 174, 105507. DOI.

> **Note**<br>This pipeline will take over 2 hours to complete. So we will not need to wait the entire duration of the pipeline to complete the lab. Follow the steps until our pipeline job has started.

This is a tabular dataset, and in our pipeline we'll use the dataset to train, evaluate, and deploy an AutoML model that classifies beans into one of 7 types based on their characteristics.

This pipeline will:

* Create a Dataset in Vertex AI.
* Train a tabular classification model with AutoML.
* Get evaluation metrics on this model.
* Based on the evaluation metrics, decide whether to deploy the model using conditional logic in Vertex Pipelines. 
* Deploy the model to an endpoint using Vertex Prediction.

Each of the steps outlined will be a component. Most of the pipeline steps will use pre-built components for Vertex AI services via the `google_cloud_pipeline_components` library we imported earlier in this codelab. In this section, we'll define one custom component first. Then, we'll define the rest of the pipeline steps using pre-built components. Pre-built components make it easier to access Vertex AI services, like model training and deployment.

The majority of time for this step is for the AutoML training piece of this pipeline, which will take **about an hour**.

## Step 1: A Custom Component For Model Evaluation

The custom component we'll define will be used towards the end of the pipeline once model training has completed. This component will do a few things:

* Get the evaluation metrics from the trained AutoML classification model.
* Parse the metrics and render them in the Vertex Pipelines UI.
* Compare the metrics to a threshold to determine whether the model should be deployed.

Before defining the component, understand its input and output parameters. As input, this pipeline takes some metadata on our Cloud project, the resulting trained model (we'll define this component later), the model's evaluation metrics, and a `thresholds_dict_str`. The `thresholds_dict_str` is something we'll define when you run your pipeline. In the case of this classification model, this will be the area under the ROC curve value for which we should deploy the model. For example, if we pass in 0.95, that means we'd only like our pipeline to deploy the model if this metric is above 95%.

The evaluation component returns a string indicating whether or not to deploy the model. Now we will create this custom component:

In [13]:
@component(
    base_image="gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest",
    output_component_file="tables_eval_component.yaml", # Optional: you can use this to load the component later
    packages_to_install=["google-cloud-aiplatform"],
)
def classif_model_eval_metrics(
    project: str,
    location: str,  # "us-central1",
    api_endpoint: str,  # "us-central1-aiplatform.googleapis.com",
    thresholds_dict_str: str,
    model: Input[Model],
    metrics: Output[Metrics],
    metricsc: Output[ClassificationMetrics],
) -> NamedTuple("Outputs", [("dep_decision", str)]):  # Return parameter.
    """This function renders evaluation metrics for an AutoML Tabular classification model.
    It retrieves the classification model evaluation generated by the AutoML Tabular training
    process, does some parsing, and uses that info to render the ROC curve and confusion matrix
    for the model. It also uses given metrics threshold information and compares that to the
    evaluation results to determine whether the model is sufficiently accurate to deploy.
    """
    import json
    import logging
    from google.cloud import aiplatform
    
    # Fetch model eval info
    def get_eval_info(client, model_name):
        
        from google.protobuf.json_format import MessageToDict
        
        response = client.list_model_evaluations(parent=model_name)
        metrics_list = []
        metrics_string_list = []
        
        for evaluation in response:
            
            print("model_evaluation")
            print(" name:", evaluation.name)
            print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
            
            metrics = MessageToDict(evaluation._pb.metrics)
            
            for metric in metrics.keys(): logging.info("metric: %s, value: %s", metric, metrics[metric])
            
            metrics_str = json.dumps(metrics)
            metrics_list.append(metrics)
            metrics_string_list.append(metrics_str)
            
        return (
            evaluation.name,
            metrics_list,
            metrics_string_list
        )
    
    # Use the given metrics threshold(s) to determine whether the model is accurate enough to deploy.
    def classification_thresholds_check(metrics_dict, thresholds_dict):
        
        for k, v in thresholds_dict.items():
            
            logging.info("k {}, v {}".format(k, v))
            
            if k in ["auRoc", "auPrc"]:  # higher is better
                if metrics_dict[k] < v:  # if under threshold, don't deploy
                    logging.info("{} < {}; returning False".format(metrics_dict[k], v))
                    return False
                
        logging.info("threshold checks passed.")
        
        return True
    
    def log_metrics(metrics_list, metricsc):
        
        test_confusion_matrix = metrics_list[0]["confusionMatrix"]
        logging.info("rows: %s", test_confusion_matrix["rows"])
        
        # log the ROC curve
        fpr = []
        tpr = []
        thresholds = []
        
        for item in metrics_list[0]["confidenceMetrics"]:
            
            fpr.append(item.get("falsePositiveRate", 0.0))
            tpr.append(item.get("recall", 0.0))
            thresholds.append(item.get("confidenceThreshold", 0.0))
            
        print(f"fpr: {fpr}")
        print(f"tpr: {tpr}")
        print(f"thresholds: {thresholds}")
        
        metricsc.log_roc_curve(fpr, tpr, thresholds)
        
        # log the confusion matrix
        annotations = []
        
        for item in test_confusion_matrix["annotationSpecs"]: annotations.append(item["displayName"])
        
        logging.info("confusion matrix annotations: %s", annotations)
        metricsc.log_confusion_matrix(
            annotations,
            test_confusion_matrix["rows"],
        )
        
        # log textual metrics info as well
        for metric in metrics_list[0].keys():
            
            if metric != "confidenceMetrics":
                
                val_string = json.dumps(metrics_list[0][metric])
                metrics.log_metric(metric, val_string)
                
        # metrics.metadata["model_type"] = "AutoML Tabular classification"
        
    logging.getLogger().setLevel(logging.INFO)
    aiplatform.init(project=project)
    
    # extract the model resource name from the input Model Artifact
    model_resource_path = model.uri.replace("aiplatform://v1/", "")
    logging.info("model path: %s", model_resource_path)
    client_options = {"api_endpoint": api_endpoint}
    
    # Initialize client that will be used to create and send requests.
    client = aiplatform.gapic.ModelServiceClient(client_options=client_options)
    eval_name, metrics_list, metrics_str_list = get_eval_info(client, model_resource_path
                                                             )
    logging.info("got evaluation name: %s", eval_name)
    logging.info("got metrics list: %s", metrics_list)
    log_metrics(metrics_list, metricsc)
    
    thresholds_dict = json.loads(thresholds_dict_str)
    deploy = classification_thresholds_check(metrics_list[0], thresholds_dict)
    dep_decision = "true" if deploy else "false"
    
    logging.info("deployment decision is %s", dep_decision)
    
    return (dep_decision,)

## Step 2: Adding Google Cloud Pre-Built Components

In this step we'll define the rest of our pipeline components and see how they all fit together.

First, define the display name for our pipeline run using a timestamp:

In [14]:
import time
DISPLAY_NAME = 'automl-beans{}'.format(str(int(time.time())))
print(DISPLAY_NAME)

automl-beans1657172479


In [15]:
@kfp.dsl.pipeline(name="automl-tab-beans-training-v2",
                  pipeline_root=PIPELINE_ROOT)
def pipeline(
    bq_source: str = "bq://aju-dev-demos.beans.beans1",
    display_name: str = DISPLAY_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    thresholds_dict_str: str = '{"auRoc": 0.95}',
):
    dataset_create_op = gcc_aip.TabularDatasetCreateOp(project=project, display_name=display_name, bq_source=bq_source)
    training_op = gcc_aip.AutoMLTabularTrainingJobRunOp(
        project=project,
        display_name=display_name,
        optimization_prediction_type="classification",
        budget_milli_node_hours=1000,
        column_transformations=[
            {"numeric": {"column_name": "Area"}},
            {"numeric": {"column_name": "Perimeter"}},
            {"numeric": {"column_name": "MajorAxisLength"}},
            {"numeric": {"column_name": "MinorAxisLength"}},
            {"numeric": {"column_name": "AspectRation"}},
            {"numeric": {"column_name": "Eccentricity"}},
            {"numeric": {"column_name": "ConvexArea"}},
            {"numeric": {"column_name": "EquivDiameter"}},
            {"numeric": {"column_name": "Extent"}},
            {"numeric": {"column_name": "Solidity"}},
            {"numeric": {"column_name": "roundness"}},
            {"numeric": {"column_name": "Compactness"}},
            {"numeric": {"column_name": "ShapeFactor1"}},
            {"numeric": {"column_name": "ShapeFactor2"}},
            {"numeric": {"column_name": "ShapeFactor3"}},
            {"numeric": {"column_name": "ShapeFactor4"}},
            {"categorical": {"column_name": "Class"}}
        ],
        dataset=dataset_create_op.outputs["dataset"],
        target_column="Class"
    )
    model_eval_task = classif_model_eval_metrics(
        project,
        gcp_region,
        api_endpoint,
        thresholds_dict_str,
        training_op.outputs["model"]
    )
    
    with dsl.Condition(model_eval_task.outputs["dep_decision"] == "true", name="deploy_decision"):
        
        deploy_op = gcc_aip.ModelDeployOp(      # noqa: F841
            model=training_op.outputs["model"],
            project=project,
            machine_type="n1-standard-4"
        )

What's happening in this code:

* First, just as in the previous pipeline, we define the input parameters this pipeline takes. We need to set these manually since they don't depend on the output of other steps in the pipeline.
* The rest of the pipeline uses a few pre-built components for interacting with Vertex AI services:|
    * `TabularDatasetCreateOp` creates a tabular dataset in Vertex AI given a dataset source either in Cloud Storage or BigQuery. 
        * In this pipeline, we're passing the data via a BigQuery table URL.
    * `AutoMLTabularTrainingJobRunOp` kicks off an AutoML training job for a tabular dataset. 
        * We pass a few configuration parameters to this component, including the model type (in this case, classification), some data on the columns, how long we'd like to run training for, and a pointer to the dataset. 
        * Notice that to pass in the dataset to this component, we're providing the output of the previous component via `dataset_create_op.outputs["dataset"]`.
    * `ModelDeployOp` deploys a given model to an endpoint in Vertex AI. 
        * There are additional configuration options available, but here we're providing the endpoint machine type, project, and model we'd like to deploy. 
        * We're passing in the model by accessing the outputs of the training step in your pipeline.
* This pipeline also makes use of conditional logic, a feature of Vertex Pipelines that lets us define a condition, along with different branches based on the result of that condition. 
    * Remember that when we defined the pipeline we passed a `thresholds_dict_str` parameter. 
        * This is the accuracy threshold we're using to determine whether to deploy our model to an endpoint. 
    * To implement this, make use of the `Condition` class from the KFP SDK. 
        * The condition passed in is the output of the custom eval component you defined earlier in this lab. 
            * If this condition is true, the pipeline will continue to execute the `deploy_op component`. 
            * If accuracy doesn't meet the predefined threshold, the pipeline will stop here and won't deploy a model.

## Step 3: Compile & Run The End-To-End ML Pipeline

With the full pipeline defined, it's time to compile it:

In [16]:
compiler.Compiler().compile(
    pipeline_func=pipeline, 
    package_path="tab_classif_pipeline.json"
)

Next, kick off a pipeline run:

In [17]:
response = api_client.create_run_from_job_spec(
    "tab_classif_pipeline.json", 
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        "project": PROJECT_ID,
        "display_name": DISPLAY_NAME
    }
)

Click on the link shown after running the cell above to see your pipeline in the console. This pipeline will take a little over an hour to run. Most of the time is spent in the AutoML training step. The completed pipeline will look something like this:

![](https://cdn.qwiklabs.com/Y%2BSqksizjHLX3Z84cImy0UwltxfTFS%2BNdDJO2OFWnNc%3D)

## Step 4: Comparing Metrics Across Pipeline Runs

If we run this pipeline multiple times, we may want to compare metrics across runs. We can use the `aiplatform.get_pipeline_df()` method to access run metadata. Here, we'll get metadata for all runs of this pipeline and load it into a Pandas DataFrame: